Imports importantes

In [1]:
import pandas as pd
import numpy as np

Carregando Dataset

In [2]:
tmdb_df = pd.read_csv("tmdb_new.csv")
tmdb_df.head()
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12020 entries, 0 to 12019
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       12020 non-null  int64  
 1   title                    12020 non-null  object 
 2   vote_average             12020 non-null  float64
 3   vote_count               12020 non-null  float64
 4   release_date             12020 non-null  object 
 5   revenue                  12020 non-null  float64
 6   runtime                  12020 non-null  float64
 7   budget                   12020 non-null  float64
 8   original_language        12020 non-null  object 
 9   popularity               12020 non-null  float64
 10  genres                   12020 non-null  object 
 11  production_companies     11618 non-null  object 
 12  production_countries     11769 non-null  object 
 13  cast                     11970 non-null  object 
 14  director              

In [ ]:
from scipy.sparse import save_npz
from scipy.sparse import csr_matrix
from collections import Counter


def str_para_categorico(df: pd.DataFrame, coluna: str, k: int) -> pd.DataFrame:
    """
    Converte atributo de string para matriz esparsa
    """
    c = Counter()
    for string in df[coluna]:
        if string is not np.nan:
            c.update(string.split(", ")) 
    res = pd.DataFrame(index=df["title"])
    
    n_cols = 0
    for i in c.most_common():
        n_cols += 1
        if i[1] < k:
            break
    print(coluna, n_cols, "colunas")

    for i in c.most_common():
        if i[1] < k:
            break
        nova_col = []
        for string in df[coluna]:
            if string is not np.nan:
                nova_col.append(1 if i[0] in string.split(", ") else 0)
            else:
                nova_col.append(0)
        res[i[0]] = np.array(nova_col, dtype=np.int8)
        res = res.copy()

    sparse = csr_matrix(res) 
    save_npz(f"matrizes_esparsas/sparse_{coluna}_{k}.npz", sparse) # Salva em forma de matriz esparsa scipy

    return res

In [ ]:
def str_para_categorico_v2(df: pd.DataFrame, coluna: str, k: int) -> pd.DataFrame:
    """
    Converte atributo de string para matriz esparsa
    """
    c = Counter()
    for string in df[coluna]:
        if string is not np.nan:
            c.update(string.split(", ")) 
    res = pd.DataFrame(index=df["title"])
    
    n_cols = 0
    for i in c.most_common():
        n_cols += 1
        if i[1] < k:
            break
    print(coluna, n_cols, "colunas")

    for i in c.most_common():
        if i[1] < k:
            break
        nova_col = []
        for string in df[coluna]:
            if string is not np.nan:
                nova_col.append(1 if i[0] in string.split(", ") else 0)
            else:
                nova_col.append(0)
        res[i[0]] = np.array(nova_col, dtype=np.int8)
        res = res.copy()

    sparse = csr_matrix(res) 
    save_npz(f"matrizes_esparsas/sparse_{coluna}_{k}.npz", sparse) # Salva em forma de matriz esparsa scipy

    return res

Criar e salvar matrizes esparsas

In [ ]:
for col in ["genres", "writers", "director", "director_of_photography", "music_composer", "cast", "production_countries", "original_language", "producers", "production_companies"]:
    try:
        str_para_categorico(tmdb_df, col, 25)
    except:
        print(col)

19 colunas
13 colunas
9 colunas
46 colunas
68 colunas
751 colunas


In [5]:
from scipy.sparse import load_npz
from scipy.sparse import hstack
import os

def abrir_matrizes_esparsas(k: int = 5):
    """
    Carrega e concatena matrizes esparsas
    """
    l = []
    for arquivo in os.listdir("./matrizes_esparsas"):
        try:
            k_ok = int(arquivo.split("_")[-1][:-4]) == k
        except:
            k_ok = False
        if arquivo[-4:] == ".npz" and k_ok:
            l.append(load_npz(f"./matrizes_esparsas/{arquivo}"))
            print(arquivo)
    return hstack(l)

In [7]:
sparse = abrir_matrizes_esparsas(25)

sparse_genres_25.npz
sparse_producers_25.npz
sparse_production_countries_25.npz
sparse_music_composer_25.npz
sparse_director_of_photography_25.npz
sparse_director_25.npz
sparse_production_companies_25.npz
sparse_writers_25.npz
sparse_cast_25.npz
sparse_original_language_25.npz


Separando Conjunto de treinamento e teste

In [8]:
from sklearn.model_selection import train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(
    hstack([csr_matrix(tmdb_df[["budget", "runtime"]]), sparse]), tmdb_df["revenue"].to_numpy(), test_size=0.2, random_state=42
    )

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.neural_network import MLPRegressor

l = RandomForestRegressor(random_state=42)

# forest ≃ 94_000_000
# votingRegressor ≃ 95_000_000
# mlp ≃ 101_000_000
# linear ≃ 102_000_000

l.fit(X_treino, y_treino)


Erro médio no conjunto de treino: 35994938.63645456
Erro médio no conjunto de val: 95965871.44893236


In [ ]:
from sklearn.metrics import root_mean_squared_error

print(f"Erro médio no conjunto de treino: {root_mean_squared_error(y_treino, l.predict(X_treino))}")
print(f"Erro médio no conjunto de teste: {root_mean_squared_error(y_teste, l.predict(X_teste))}")

In [ ]:
# Salvando modelo

import pickle

with open("modelos/random_forest_revenue_k=25.pkl", "wb") as f:
    pickle.dump(l, f)

In [ ]:
# Carregando modelo

with open("modelos/lasso-imdb_rating.pkl", "rb") as f:
    l = pickle.load(f)

Testando diferentes entradas

In [16]:
for col in ["genres", "writers", "director", "director_of_photography", "music_composer", "cast", "production_countries", "original_language", "producers", "production_companies"]:
    try:
        str_para_categorico(tmdb_df, col, 10)
    except:
        print(col)

19 colunas
243 colunas
146 colunas
258 colunas
219 colunas
4782 colunas
60 colunas
40 colunas
948 colunas
539 colunas


In [17]:
from sklearn.metrics import root_mean_squared_error

for col in ["genres", "writers", "director", "director_of_photography", "music_composer", "cast", "production_countries", "original_language", "producers", "production_companies"]:
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(
        hstack([csr_matrix(tmdb_df[["budget", "runtime"]]), load_npz(f"matrizes_esparsas/sparse_{col}_10.npz")]), tmdb_df["revenue"].to_numpy(), test_size=0.2, random_state=13
    )
    from sklearn.ensemble import RandomForestRegressor

    l = RandomForestRegressor(random_state=42)

    l.fit(X_treino, y_treino)

    print(f"{col:<25}: {root_mean_squared_error(y_teste, l.predict(X_teste)):9.3f}")

genres                   : 108441173.698
writers                  : 103960874.948
director                 : 111717593.803
director_of_photography  : 109383998.408
music_composer           : 103768307.644
cast                     : 98270671.449
production_countries     : 115452040.997
original_language        : 117723900.328
producers                : 100167599.362
production_companies     : 100558968.693
